In [116]:
import py2cytoscape.cytoscapejs as renderer
import random
import requests
import sys
import networkx as nx
import pprint

from py2cytoscape.data.cyrest_client import CyRestClient

#args = sys.argv[1:]

seed = 5

random.seed(seed)

graph_size = 50
graph_size_range = graph_size / 2

edge_num_limit = 2
hub_node_prob = 0.2

num_v = random.randrange(graph_size - graph_size_range, graph_size + graph_size_range)

g = nx.MultiDiGraph()

num_i_hub = 0
num_o_hub = 0

for i in range(num_v):
    i_hub = False
    o_hub = False
    if random.random() < hub_node_prob:
        i_hub = True
        num_i_hub += 1
    #if random.random() < hub_node_prob:
    #    o_hub = True
    #    num_o_hub += 1
    g.add_node(i, i_hub=i_hub, o_hub=o_hub)

for x in g:
    tmp = []
    e_min = 1
    e_max = edge_num_limit
    if g.nodes[x]['o_hub']:
        e_min = edge_num_limit * 2
        e_max = edge_num_limit * 3
    for i in range(random.randrange(e_min, e_max)):
        y = random.randrange(0, num_v)
        if x != y and y not in tmp:
            tmp.append(y)
            g.add_edge(x,y)

for y in g:
    if not g.nodes[y]['i_hub']:
        continue
    for i in range(edge_num_limit):
        x = random.randrange(0, num_v)
        if g.has_edge(x,y):
            continue
        if x == y:
            continue
        g.add_edge(x,y)

for i in g:
    n = g.nodes[i]
    if n['i_hub'] and n['o_hub']:
        n['color'] = 'red'
    elif n['i_hub']:
        n['color'] = 'blue'
    elif n['o_hub']:
        n['color'] = 'grey'
    else:
        n['color'] = 'grey'

for e in g.edges.data():
    x = e[0]
    y = e[1]
    attr = e[2]
    if g.nodes[x]['o_hub'] and g.nodes[y]['i_hub']:
        attr['color'] = 'red'
    elif g.nodes[x]['o_hub']:
        attr['color'] = 'grey'
    elif g.nodes[y]['i_hub']:
        attr['color'] = 'blue'
    else:
        attr['color'] = 'grey'

print('Number of Nodes: ' + str(g.number_of_nodes()))
print('Number of Edges: ' + str(g.number_of_edges()))
print('Number of in-hub: ' + str(num_i_hub))
print('Number of out-hub: ' + str(num_o_hub))

Number of Nodes: 64
Number of Edges: 94
Number of in-hub: 16
Number of out-hub: 0


In [117]:
from pyvis.network import Network

net = Network(directed=True, notebook=True, height='700px', width = '100%')

net.from_nx(g)

#net.toggle_physics(False)

net.show('graph.html')

In [ ]:
cy = CyRestClient()
nw = cy.network.create(name='My Network', collection='My network collection')

g_cy = cy.network.create_from_networkx(g)

layout = cy.style.create('Directed')
cy.layout.apply(name='cose', network=g_cy)
cy.style.apply(layout, network=g_cy)
cy.edgebundling.apply(g_cy)

In [47]:
default_style = cy.style.create('default')

new_defaults = {
    # Node defaults
    'NODE_BORDER_PAINT': '#919191',
    'NODE_FILL_COLOR': 'white',
    'NODE_SIZE': 25,
    'NODE_BORDER_WIDTH': 3,
    'NODE_TRANSPARENCY': 255,
    'NODE_LABEL_COLOR': '#3399FF',
    'NODE_SHAPE': 'Ellipse',
    
    # Edge defaults
    'EDGE_COLOR': '#919191',
    'EDGE_WIDTH': 3,
    'EDGE_STROKE_UNSELECTED_PAINT': '#919191',
    'EDGE_TARGET_ARROW_UNSELECTED_PAINT': '#919191',
    'EDGE_LINE_TYPE': 'EQUAL_DASH',
    'EDGE_TRANSPARENCY': 255,
    'EDGE_TARGET_ARROW_SHAPE': 'ARROW',
    
    # Network defaults
    'NETWORK_BACKGROUND_PAINT': 'white'
}

default_style.update_defaults(new_defaults)

cy.style.apply(default_style, network=g_cy)

In [6]:
print(view)

{'data': {'__Annotations': [], 'selected': True, 'shared_name': 'From cyREST', 'name': 'From cyREST', 'SUID': 27066}, 'generated_by': 'cytoscape-3.8.2', 'format_version': '1.0', 'elements': {'edges': [{'selected': False, 'data': {'BEND_MAP_ID': 27120, 'selected': False, 'shared_interaction': '0', 'SUID': 27120, 'interaction': '0', 'source': '27097', 'target_original': '4', 'id': '27120', 'target': '27080', 'source_original': '21'}}, {'selected': False, 'data': {'BEND_MAP_ID': 27119, 'selected': False, 'shared_interaction': '0', 'SUID': 27119, 'interaction': '0', 'source': '27095', 'target_original': '15', 'id': '27119', 'target': '27091', 'source_original': '19'}}, {'selected': False, 'data': {'BEND_MAP_ID': 27118, 'selected': False, 'shared_interaction': '0', 'SUID': 27118, 'interaction': '0', 'source': '27095', 'target_original': '21', 'id': '27118', 'target': '27097', 'source_original': '19'}}, {'selected': False, 'data': {'BEND_MAP_ID': 27117, 'selected': False, 'shared_interaction